# Preprocessing Data

In [1]:
import os.path
import os
import pandas as pd
import numpy as np
import censusgeocode as cg
path = os.path.abspath(os.getcwd())
data_path = os.path.abspath(os.path.join(path,'Data'))

In [2]:
def geocode(stop_lat, stop_lon):
    """
    
    :param stop_lat:
    :param stop_lon:
    """
    results = cg.coordinates(x=stop_lon, y=stop_lat)['2010 Census Blocks']
    return results[0]['BLOCK']

In [9]:
blocks = []
for ind,tup in stops.iterrows():
    results = cg.coordinates(x=tup.stop_lon,y=tup.stop_lat)
    blocks.append(results)
blocks

ValueError: Unable to parse response from Census

In [10]:
stops = pd.read_csv(os.path.join(data_path,'stops.txt'))
# generating data
stop_ids = stops['stop_id'].to_numpy()
gen = 1000* np.random.normal(size=stop_ids.shape)
gen = np.int32(10000 + np.floor(gen))
stop_ids = pd.DataFrame(data = [stop_ids,gen]).T
stop_ids.columns = ['stop_id','rides']
stops = stops.merge(stop_ids)
#stops.insert(len(stops.columns),'Block',stops.apply(lambda x: geocode(x.stop_lat,x.stop_lon),axis=1))
#stops.insert(len(stops.columns),'Block',2013)
stops.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,rides
0,100148,100148,MARTIN L KING JR DR NW @ CHICAMAUGA AVE NW,33.753823,-84.431838,7807
1,210829,99971061,DEERFIELD PKWY @ TWO VERIZON PL,34.092878,-84.271101,9781
2,100150,100150,MARTIN L KING JR DR NW @ CHAPPELL RD,33.753841,-84.433273,9895
3,100152,100152,FAIR ST SW @ 1ST ST SW,33.748905,-84.425545,10169
4,100154,100154,FAIR ST@FIRST ST,33.749077,-84.425298,10180


In [11]:
stops.shape

(9170, 6)

In [38]:
census = pd.read_csv(os.path.join(data_path,'census_2010.csv'))
census = census.merge(stops,on="Block")

In [27]:
census.columns

Index(['Area Name-Legal/Statistical Area Description', 'Qualifying Name',
       'Area (Land)', 'Area (Water)', 'Summary Level', 'Geographic Component',
       'Region', 'Division', 'FIPS', 'State (FIPS)', 'County', 'Census Tract',
       'Block', 'Total Population', 'Total Population.1',
       'Total Population: Male', 'Total Population: Male: Under 5 years',
       'Total Population: Male: 5 to 9 years',
       'Total Population: Male: 10 to 14 years',
       'Total Population: Male: 15 to 17 years',
       'Total Population: Male: 18 to 24 years',
       'Total Population: Male: 25 to 34 years',
       'Total Population: Male: 35 to 44 years',
       'Total Population: Male: 45 to 54 years',
       'Total Population: Male: 55 to 64 years',
       'Total Population: Male: 65 and 74 years',
       'Total Population: Male: 75 to 84 years',
       'Total Population: Male: 85 years and over', 'Total Population: Female',
       'Total Population: Female: Under 5 years',
       'Total Pop

In [28]:
pd.unique(census.County)

array([ 63,  89, 121])